In [99]:
import numpy as np
import pandas as pd

In [100]:
df=pd.read_csv('spam.csv',encoding='latin-1')

In [101]:
df.head()

,v1,v2,Unnamed: 2,Unnamed: 3,Unnamed: 4
0,ham,"Go until jurong point, crazy.. Available only ...",NaN,NaN,NaN
1,ham,Ok lar... Joking wif u oni...,NaN,NaN,NaN
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,NaN,NaN,NaN
3,ham,U dun say so early hor... U c already then say...,NaN,NaN,NaN
4,ham,"Nah I don't think he goes to usf, he lives aro...",NaN,NaN,NaN


In [102]:
df.drop(columns=['Unnamed: 2','Unnamed: 3','Unnamed: 4'],inplace=True)

In [103]:
df.head()

,v1,v2
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


# Step 2: EDA

In [104]:
df['v1']=df['v1'].map({'spam':1,'ham':0})


In [105]:
df.head()

,v1,v2
0,0,"Go until jurong point, crazy.. Available only ..."
1,0,Ok lar... Joking wif u oni...
2,1,Free entry in 2 a wkly comp to win FA Cup fina...
3,0,U dun say so early hor... U c already then say...
4,0,"Nah I don't think he goes to usf, he lives aro..."


In [106]:
df.isnull().sum()

v1    0
v2    0
dtype: int64

In [107]:
df.shape

(5572, 2)

In [108]:
import nltk
nltk.download('stopwords')
import string
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import remove_stopwords

stop_words = set(stopwords.words('english'))

def preprocess(s):
    # 1. Convert to lowercase
    s = s.lower()
    
    # 2. Remove punctuation
    s = s.translate(str.maketrans('', '', string.punctuation))
    
    # 3. Tokenize using gensim's simple_preprocess
    tokens = simple_preprocess(s, deacc=True)  # deacc=True removes punctuation as well
    
    # 4. Remove stopwords using gensim
    tokens = [word for word in tokens if word not in stop_words]
    
    # 5. Join tokens back into a single string
    processed_text = ' '.join(tokens)
    
    return processed_text


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\tanee\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [109]:
df['v2_new']=df['v2'].apply(preprocess)

In [110]:
df.head()

,v1,v2,v2_new
0,0,"Go until jurong point, crazy.. Available only ...",go jurong point crazy available bugis great wo...
1,0,Ok lar... Joking wif u oni...,ok lar joking wif oni
2,1,Free entry in 2 a wkly comp to win FA Cup fina...,free entry wkly comp win fa cup final tkts st ...
3,0,U dun say so early hor... U c already then say...,dun say early hor already say
4,0,"Nah I don't think he goes to usf, he lives aro...",nah dont think goes usf lives around though


# Step 3: Train model

In [111]:
from sklearn.model_selection import train_test_split
x=df['v2_new']
y=df['v1']
X_train,X_test,y_train,y_test=train_test_split(x,y,test_size=0.2,random_state=42)

In [112]:
from sklearn.metrics import classification_report
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC


In [113]:
# Use CountVectorizer to vectorize the text data
from sklearn.feature_extraction.text import CountVectorizer

count_vectorizer = CountVectorizer(stop_words='english', max_df=0.95)
X_train_vectorize = count_vectorizer.fit_transform(X_train)
X_test_vectorize = count_vectorizer.transform(X_test)

In [114]:
#1. MultinomialNB
nb_model = MultinomialNB()
nb_model.fit(X_train_vectorize, y_train)
y_pred_nb = nb_model.predict(X_test_vectorize)
nb_report = classification_report(y_test, y_pred_nb)
print("Naive Bayes Report:")
print(nb_report)

Naive Bayes Report:
              precision    recall  f1-score   support

           0       0.99      0.99      0.99       965
           1       0.96      0.91      0.94       150

    accuracy                           0.98      1115
   macro avg       0.98      0.95      0.96      1115
weighted avg       0.98      0.98      0.98      1115



In [115]:
# 2. Logistic Rgression
lr_model = LogisticRegression(max_iter=1000, random_state=42)
lr_model.fit(X_train_vectorize, y_train)
y_pred_lr = lr_model.predict(X_test_vectorize)
lr_report = classification_report(y_test, y_pred_lr)
print("Logistic Regression Report:")
print(lr_report)

Logistic Regression Report:
              precision    recall  f1-score   support

           0       0.97      1.00      0.99       965
           1       0.99      0.83      0.90       150

    accuracy                           0.98      1115
   macro avg       0.98      0.91      0.94      1115
weighted avg       0.98      0.98      0.97      1115



In [116]:
#3.Support Vector Machine classifier
svm_model = SVC(random_state=42)
svm_model.fit(X_train_vectorize, y_train)
y_pred_svm = svm_model.predict(X_test_vectorize)
svm_report = classification_report(y_test, y_pred_svm)
print("SVM Report:")
print(svm_report)

SVM Report:
              precision    recall  f1-score   support

           0       0.97      1.00      0.99       965
           1       0.99      0.82      0.90       150

    accuracy                           0.97      1115
   macro avg       0.98      0.91      0.94      1115
weighted avg       0.98      0.97      0.97      1115



# Use TF-IDF to vectorize the text data

In [117]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf_vectorizer = TfidfVectorizer(stop_words='english', max_df=0.95)
X_train_tfidf = tfidf_vectorizer.fit_transform(X_train)
X_test_tfidf = tfidf_vectorizer.transform(X_test)

In [118]:
#1. MultinomialNB
nb_model = MultinomialNB()
nb_model.fit(X_train_tfidf, y_train)
y_pred_nb = nb_model.predict(X_test_tfidf)
nb_report = classification_report(y_test, y_pred_nb)
print("Naive Bayes Report:")
print(nb_report)

Naive Bayes Report:
              precision    recall  f1-score   support

           0       0.96      1.00      0.98       965
           1       1.00      0.76      0.86       150

    accuracy                           0.97      1115
   macro avg       0.98      0.88      0.92      1115
weighted avg       0.97      0.97      0.97      1115



In [119]:
# 2. Logistic Rgression
lr_model = LogisticRegression(max_iter=1000, random_state=42)
lr_model.fit(X_train_tfidf, y_train)
y_pred_lr = lr_model.predict(X_test_tfidf)
lr_report = classification_report(y_test, y_pred_lr)
print("Logistic Regression Report:")
print(lr_report)

Logistic Regression Report:
              precision    recall  f1-score   support

           0       0.95      1.00      0.97       965
           1       0.97      0.66      0.79       150

    accuracy                           0.95      1115
   macro avg       0.96      0.83      0.88      1115
weighted avg       0.95      0.95      0.95      1115



In [120]:
#3.Support Vector Machine classifier
svm_model = SVC(random_state=42)
svm_model.fit(X_train_tfidf, y_train)
y_pred_svm = svm_model.predict(X_test_tfidf)
svm_report = classification_report(y_test, y_pred_svm)
print("SVM Report:")
print(svm_report)

SVM Report:
              precision    recall  f1-score   support

           0       0.97      1.00      0.98       965
           1       0.98      0.81      0.89       150

    accuracy                           0.97      1115
   macro avg       0.98      0.90      0.94      1115
weighted avg       0.97      0.97      0.97      1115

